# Bargain Bot

Goal is to make a bot that can perform bargain with a real human and make deals. There are couple of things here:

 - Understanding human's input and get context out of it.
 - If the context matches the goal, see if it can fulfill bot's goal in short or long term.
 - Learn the process of bargaining
 - Generate the answer back to human
 - It the input is out of context, try and give a relevant answer.
 
### Primary Goal: Understand different aspects of NLP, try different existing algorithms and build a service which can interact with universe in natural language and fulfil it's goals *alomost* from scratch.

### Will start with traditional algorithms and eventually move to deep learning techniques

### Experiment 1: Understanding
#### I'll give you four bananas if you give me two mangos.

This is one of the use case which BargainBot should be able to deal with; It should be able to understand the statement, check if it's valid in given context; Ask for more information if need be; Check if it meet's it's own goals and if everything is cool, make the deal and make necessary changes;

While interacting with the user it should be able to generate appropriate response and try to be as less bot as possible here;

In [1]:
input = "I'll give you four bananas if you give me two mangos."

In [2]:
# setting up the environment
file_names=['gs', 'gswin32c.exe', 'gswin64c.exe']
%xmode verbose

import math
import random
from time import time

import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer

Exception reporting mode: Verbose


In [3]:
# tokenize a sentence to tokens
tokens = nltk.word_tokenize(input)
print (tokens)

['I', "'ll", 'give', 'you', 'four', 'bananas', 'if', 'you', 'give', 'me', 'two', 'mangos', '.']


In [4]:
# let's look at the POS Tags generated by NLTK toolkit
tagged = nltk.pos_tag(tokens)
print (tagged)
print ()

# tree representation
entities = nltk.chunk.ne_chunk(tagged)
print ( entities.__repr__())

[('I', 'PRP'), ("'ll", 'MD'), ('give', 'VB'), ('you', 'PRP'), ('four', 'CD'), ('bananas', 'NNS'), ('if', 'IN'), ('you', 'PRP'), ('give', 'VBP'), ('me', 'PRP'), ('two', 'CD'), ('mangos', 'NNS'), ('.', '.')]

Tree('S', [('I', 'PRP'), ("'ll", 'MD'), ('give', 'VB'), ('you', 'PRP'), ('four', 'CD'), ('bananas', 'NNS'), ('if', 'IN'), ('you', 'PRP'), ('give', 'VBP'), ('me', 'PRP'), ('two', 'CD'), ('mangos', 'NNS'), ('.', '.')])


# let's define the universe for this bot;
Every bot will have a universe where known where we will define all the rules for the bot; All entities it knows, initialize the params to some values;

As of now it has following things:
 - Objects: Entities known to the bot, no of the entity the bot has and how much it likes the item relatively;

In [5]:
random.seed(time())

class Universe:
    def initCount(self, maxVal):
        return math.floor((random.random() * 1000 * maxVal) % maxVal)

    def __init__(self):
        # defining the objects the bot has and how much fuck it gives about it. 
        self.entities = {"mango": [self.initCount(100), 1], "banana": [self.initCount(100), 2]}
        
        print ("Initialized universe")
        for k, v in self.entities.items():
            print ("%s: %d, pref: %d" % (k, v[0], v[1]))
            
universe = Universe()

Initialized universe
mango: 98, pref: 1
banana: 36, pref: 2


## Frames
Let's define a frame which will be the basic unit for arithmetic operation from bot's perspective; It will contain properties like:
 - Operation: Addition | Subtraction | maybe Multiplication & Division
 - Entity: Entity under operation
 - Count: No of entities under consideration

In future we can add some fancy properties like `interest score`, `charm`,  `relationship factors` etc when bots interact with other bots or humans as well.

In [7]:
class Frame:
    def __init__(self):
        self.operation = None
        self.entity = None
        self.count = None
    
    def SetOperation(self, operation):
        self.operation = operation

    def SetEntity(self, entity):
        self.entity = entity

    def SetCount(self, count):
        self.count = count
        
    def IsValid(self):
        return self.entity and self.operation and self.count
    
    def ToString(self, printInValid = False):
        if (not self.IsValid() and not printInValid):
            return "Invalid Frame"

        _return = ""
        _return += ("OPERATION: %s" % self.operation) +"\n"
        _return += ("Entity: %s" % self.entity) +"\n"
        _return += ("Count: %s" % self.count) +"\n"
        return _return

 
# Starting with a very NAIVE Approach
class FrameFactory:
    def __init__(self, universe):
        self.universe = universe
        self.stemmer = SnowballStemmer("english")
        self.entities = {}
        for k, v in self.universe.entities.items():
            self.entities[self.stemmer.stem(k)] = k
         
        # TODO: find some library that deals with this
        self.numbers = {"one": 1, "a": 1, "two": 2, "three": 3, "four": 4, "five": 5, "six": 6,
                        "seven": 7, "eight": 8, "nine": 9, "ten" : 10}
        
#         self.__inner = {
#             "BOT": ["you", "bot", "bargainbot"],
#             "USER": ["i", "me"],
#             "PVERB": ["borrow", "take"],
#             "NVERB": ["give", "lend"],
#             "COUNT": [i for in in self.numbers.items()],
#             "ENTITIES": [i for i in self.entities.items()],
#         }
        
    def ToFrames(self, sentence):
        # a sentence can have one or more frames
        # [can] [you] -> p1 [give] -> o1 [me] -> p2
        # [one] -> count [apple] -> entity
        # assuming one frame per sentence
        frame = Frame()
        
        tree = nltk.chunk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sentence.lower())))
        for word, wordType in tree:
            word = self.stemmer.stem(word)
#             print (word, wordType)
            if (wordType in ['NNS', 'NN']): 
                if word in self.entities:
                    frame.SetEntity(word)
                else:
                    break

            if (wordType == 'PRP' and word == 'you'): frame.SetOperation("ADD")
            elif(wordType == 'PRP' and word == 'me'): frame.SetOperation("SUB")

            if (wordType in ['CD','VB','DT'] and word in self.numbers):
                frame.SetCount(self.numbers[word])
            elif (wordType == 'CD'):
                count = None
                break
            
        return [frame]

## Lets start with easier statements and approach
sentence1 = "I'll give you two bananas"
sentence2 = "you give me three mangos"
sentence3 = "give me a bananas"

frameFactory = FrameFactory(universe)

print ("Statement: ", sentence1)
print (frameFactory.ToFrames(sentence1)[0].ToString(True))

print ("Statement: ", sentence2)
print (frameFactory.ToFrames(sentence2)[0].ToString(True))

print ("Statement: ", sentence3)
print (frameFactory.ToFrames(sentence3)[0].ToString(True))

Statement:  I'll give you two bananas
OPERATION: None
Entity: None
Count: None

Statement:  you give me three mangos
OPERATION: SUB
Entity: mango
Count: 3

Statement:  give me a bananas
OPERATION: SUB
Entity: banana
Count: 1



#### Here we are able to convert a single simple statement to a Frame - using POS tags, without considering the sequence of tokens
Now a frame can be used by the bot to do the math, take logical actions, decide to deal or not by performing corresponding math;

But yeah We need to have logic to convert complext sentences to an array of frames and perform complete math. The sequence of tokens matter for this kind of operation. I'll however park that for now and continue with the naive assumption that sequence doesn't matter;

### Acting on set of frames

In [8]:
# Extend the universe to have more features
random.seed(time())

class Universe:
    def initCount(self, maxVal):
        return math.floor((random.random() * 1000 * maxVal) % maxVal)

    def __init__(self):
        # defining the entities the bot has and how much fuck it gives about it. 
        self.entities = {"mango": [self.initCount(100), 1], "banana": [self.initCount(100), 2]}
        
        print ("Initialized universe")
        print (self.ToString())
      
    def ToString(self):
        _return = ("UNIVERSE CONFIG:") +"\n"
        for k, v in self.entities.items():
            _return += ("%s: %d, pref: %d" % (k, v[0], v[1]))  +"\n"
        return _return

    def GetScore(self):
        return np.sum(np.array([j[0] * j[1] for i, j in self.entities.items()]))
    
    # method to check if Bot has the entity in asked amount
    def HaveEntity(self, entity, count = 0):
        if not entity in self.entities:
            return False
        if count > self.entities[entity][0]:
            return False
        return True
    
    def AddEntity(self, entity, count = 0):
        if entity not in self.entities:
            raise "Unknown entity: %s" % entity
        
        if self.entities[entity][0] + count < 0:
            raise "Entity: %s not available enough" % entity

        self.entities[entity][0] += count
            
universe = Universe()

def act(universe, frame):
#     print (frame.ToString())
    currentscore = universe.GetScore()
    # TODO: this should be an enum
    if frame.operation == "ADD":
        universe.AddEntity(frame.entity, frame.count)
        print ("Thanks for %d %ss" % (frame.count, frame.entity))

    if frame.operation == "SUB":
        if not universe.HaveEntity(frame.entity, frame.count):
            print ("I don't have %s in appropriate quantity" % (frame.entity))
            return
        
        universe.AddEntity(frame.entity, -frame.count)
        print ("Giving you %d %s" % (frame.count, frame.entity))
        
    print (universe.ToString())

Initialized universe
UNIVERSE CONFIG:
mango: 7, pref: 1
banana: 52, pref: 2



In [9]:
frameFactory = FrameFactory(universe)

print ("Statement: ", sentence1)
act (universe, frameFactory.ToFrames(sentence1)[0])

print ("Statement: ", sentence2)
act (universe, frameFactory.ToFrames(sentence2)[0])

Statement:  I'll give you two bananas
UNIVERSE CONFIG:
mango: 7, pref: 1
banana: 52, pref: 2

Statement:  you give me three mangos
Giving you 3 mango
UNIVERSE CONFIG:
mango: 4, pref: 1
banana: 52, pref: 2



In [10]:
# let's try one simple example
act (universe, frameFactory.ToFrames("give me two mangos")[0])
act (universe, frameFactory.ToFrames("fine I'll give you three mangos")[0])
act (universe, frameFactory.ToFrames("give me ten bananas")[0])
act (universe, frameFactory.ToFrames("give me a bananas")[0])
act (universe, frameFactory.ToFrames("giving you five bananas")[0])
act (universe, frameFactory.ToFrames("give me five mango")[0])

Giving you 2 mango
UNIVERSE CONFIG:
mango: 2, pref: 1
banana: 52, pref: 2

UNIVERSE CONFIG:
mango: 2, pref: 1
banana: 52, pref: 2

Giving you 10 banana
UNIVERSE CONFIG:
mango: 2, pref: 1
banana: 42, pref: 2

Giving you 1 banana
UNIVERSE CONFIG:
mango: 2, pref: 1
banana: 41, pref: 2

Thanks for 5 bananas
UNIVERSE CONFIG:
mango: 2, pref: 1
banana: 46, pref: 2

I don't have mango in appropriate quantity


## This marks the end of first experiment; BargainBot is a slave now, will accept the bargain if it understands your simple request;

In next experiment the goal will be to: ** Understand complex multi frame statements and accept if it's possible in universe.**

Goals after that:
 - Make greedy bargains with short term goal of minimizing loss in current bargain.
 - Ability to generate response using HMM or some other generative model
 - Learning to Bargain with long term goals, building relationship with user
 - Understanding sentiments in the conversations and take intelligent actions on long term goals.
 - Outbound bargains, estimating preference of user / other bots. Learning to deal.
 - Greedy Bargains to Reasonable Bargains (outbound) using reinforcements.
 - Bluffing, learning to bluff, learn to detect getting bluffed.
 - Participating in acutions
 - Mutli Bot scenario
 - Responses in Game of Throne Style
 
 